# Vietnamese Question Answering with BARTpho-word-base

The question answering we are developing is called **extractive question answering**, which involves posing a question about a document and indentifying answers as spans of text in the document itself. We use the UIT-ViQuAD2.0 dataset, which is the extractive question answering dataset. Other types of question answering includes **generative question answering** which generates answers for open-ended questions.

## Preparing the data

### Download UIT-ViQuAD2.0 dataset

In [ ]:
# download datasets module
!pip install datasets
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("taidng/UIT-ViQuAD2.0")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.04k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/5.20M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/735k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/28454 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3814 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7301 [00:00<?, ? examples/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'uit_id', 'title', 'context', 'question', 'answers', 'is_impossible', 'plausible_answers'],
        num_rows: 28454
    })
    validation: Dataset({
        features: ['id', 'uit_id', 'title', 'context', 'question', 'answers', 'is_impossible', 'plausible_answers'],
        num_rows: 3814
    })
    test: Dataset({
        features: ['id', 'uit_id', 'title', 'context', 'question', 'answers', 'is_impossible', 'plausible_answers'],
        num_rows: 7301
    })
})

### Print first element of the training set

In [ ]:
print("Context: ", raw_datasets["train"][0]["context"])
print("Question: ", raw_datasets["train"][0]["question"])
print("Answer: ", raw_datasets["train"][0]["answers"])

Context:  Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4 năm 2000) là Thủ tướng đầu tiên của nước Cộng hòa Xã hội chủ nghĩa Việt Nam từ năm 1976 (từ năm 1981 gọi là Chủ tịch Hội đồng Bộ trưởng) cho đến khi nghỉ hưu năm 1987. Trước đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa từ năm 1955 đến năm 1976. Ông là vị Thủ tướng Việt Nam tại vị lâu nhất (1955–1987). Ông là học trò, cộng sự của Chủ tịch Hồ Chí Minh. Ông có tên gọi thân mật là Tô, đây từng là bí danh của ông. Ông còn có tên gọi là Lâm Bá Kiệt khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm (Chủ nhiệm là Hồ Học Lãm).
Question:  Tên gọi nào được Phạm Văn Đồng sử dụng khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm?
Answer:  {'text': ['Lâm Bá Kiệt'], 'answer_start': [507]}


### During training, there is only **one** possible answer. We have to filter datasets which has only **one** possible answer. For validation and test sets, we don't need to filter because we will calculate the score between a predicted answer with all the acceptable answers and take the best score.

In [ ]:
raw_datasets['train'] = raw_datasets["train"].filter(lambda x: len(x["answers"]["text"]) == 1)

Filter:   0%|          | 0/28454 [00:00<?, ? examples/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'uit_id', 'title', 'context', 'question', 'answers', 'is_impossible', 'plausible_answers'],
        num_rows: 19238
    })
    validation: Dataset({
        features: ['id', 'uit_id', 'title', 'context', 'question', 'answers', 'is_impossible', 'plausible_answers'],
        num_rows: 3814
    })
    test: Dataset({
        features: ['id', 'uit_id', 'title', 'context', 'question', 'answers', 'is_impossible', 'plausible_answers'],
        num_rows: 7301
    })
})

## Data preprocessing

### Preprocessing the training data

#### Convert the input text to IDs

In [ ]:
from transformers import AutoTokenizer, PreTrainedTokenizerFast

model_checkpoint = "vinai/bartpho-word-base"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_checkpoint)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PhobertTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


##### Pass the question and answer to the tokenizer, it will properly insert the special tokens to form the sentence: [CLS] quesion [SEP] context [SEP]

In [ ]:
# Try tokenizer with 1 sample
context = raw_datasets["train"][0]["context"]
question = raw_datasets["train"][0]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

'<s>Tên gọi nào được Phạm Văn Đồng sử dụng khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm? </s></s>Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4 năm 2000) là Thủ tướng đầu tiên của nước Cộng hòa Xã hội chủ nghĩa Việt Nam từ năm 1976 (từ năm 1981 gọi là Chủ tịch Hội đồng Bộ trưởng) cho đến khi nghỉ hưu năm 1987. Trước đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa từ năm 1955 đến năm 1976. Ông là vị Thủ tướng Việt Nam tại vị lâu nhất (1955–1987). Ông là học trò, cộng sự của Chủ tịch Hồ Chí Minh. Ông có tên gọi thân mật là Tô, đây từng là bí danh của ông. Ông còn có tên gọi là Lâm Bá Kiệt khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm (Chủ nhiệm là Hồ Học Lãm). </s>'

##### Limit the length to avoid too long contexts. We will dealt with long context by creating several training features from one sample of our dataset, with a sliding window between them

In [ ]:
inputs = tokenizer(
    question,
    context,
    max_length=100,   # maximum length of context
    truncation="only_second",   # truncate the context (which is in the second position) when the question with its context is too long
    stride=50,        # the number of overlapping tokens between two successive chunks
    return_overflowing_tokens=True,   # use overflow tokens for the model
    return_offsets_mapping=True,    # map the tokens back to their positions in the context (the index of the token in the context where the answer starts and the index of the token where the answer ends.)
)

for ids in inputs["input_ids"]:
  print(tokenizer.decode(ids))

<s>Tên gọi nào được Phạm Văn Đồng sử dụng khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm? </s></s>Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4 năm 2000) là Thủ tướng đầu tiên của nước Cộng hòa Xã hội chủ nghĩa Việt Nam từ năm 1976 (từ năm 1981 gọi là Chủ tịch Hội đồng Bộ trưởng) cho đến khi nghỉ hưu năm 1987. Trước đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân chủ </s>
<s>Tên gọi nào được Phạm Văn Đồng sử dụng khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm? </s></s>nước Cộng hòa Xã hội chủ nghĩa Việt Nam từ năm 1976 (từ năm 1981 gọi là Chủ tịch Hội đồng Bộ trưởng) cho đến khi nghỉ hưu năm 1987. Trước đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa từ năm 1955 đến năm 1976. Ông là vị Thủ tướng Việt Nam tại vị lâu nhất (</s>
<s>Tên gọi nào được Phạm Văn Đồng sử dụng khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm? </s></s>đồng Bộ trưởng) cho đến khi nghỉ hưu năm 1987. Trước đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân chủ Cộ

In [ ]:
inputs

{'input_ids': [[0, 4473, 328, 142, 11, 6713, 2965, 2316, 5717, 8410, 26, 47, 268, 286, 9393, 2209, 2665, 47335, 61, 1710, 35, 9777, 34281, 1387, 114, 2, 2, 6713, 2965, 2316, 14157, 99, 78, 107, 29, 20043, 903, 1841, 78, 163, 29, 39587, 19, 8, 11639, 3110, 127, 6937, 7, 58, 11918, 59763, 1517, 3074, 1093, 286, 1645, 350, 590, 39, 29, 7052, 14157, 39, 29, 6885, 328, 8, 3698, 26865, 792, 80, 125, 54913, 1701, 19, 13, 30, 26, 915, 2469, 29, 12515, 36540, 347, 37, 46, 150, 300, 1766, 178, 11639, 3110, 1008, 2149, 350, 590, 2446, 286, 2], [0, 4473, 328, 142, 11, 6713, 2965, 2316, 5717, 8410, 26, 47, 268, 286, 9393, 2209, 2665, 47335, 61, 1710, 35, 9777, 34281, 1387, 114, 2, 2, 58, 11918, 59763, 1517, 3074, 1093, 286, 1645, 350, 590, 39, 29, 7052, 14157, 39, 29, 6885, 328, 8, 3698, 26865, 792, 80, 125, 54913, 1701, 19, 13, 30, 26, 915, 2469, 29, 12515, 36540, 347, 37, 46, 150, 300, 1766, 178, 11639, 3110, 1008, 2149, 350, 590, 2446, 286, 11918, 59763, 1517, 39, 29, 10850, 30, 29, 13652, 26395

In [ ]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [ ]:
inputs["overflow_to_sample_mapping"]  # divide long contexts into spans of context

[0, 0, 0, 0, 0]

In [ ]:
# Try tokenizer with more samples
inputs = tokenizer(
    raw_datasets["train"][2:6]["question"],
    raw_datasets["train"][2:6]["context"],
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

for ids in inputs["input_ids"]:
  print(tokenizer.decode(ids))

inputs["overflow_to_sample_mapping"]  # divide long contexts into spans of context

<s>Giai đoạn năm 1955-1976, Phạm Văn Đồng nắm giữ chức vụ gì? </s></s>Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4 năm 2000) là Thủ tướng đầu tiên của nước Cộng hòa Xã hội chủ nghĩa Việt Nam từ năm 1976 (từ năm 1981 gọi là Chủ tịch Hội đồng Bộ trưởng) cho đến khi nghỉ hưu năm 1987. Trước đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa từ năm 1955 </s>
<s>Giai đoạn năm 1955-1976, Phạm Văn Đồng nắm giữ chức vụ gì? </s></s>chủ nghĩa Việt Nam từ năm 1976 (từ năm 1981 gọi là Chủ tịch Hội đồng Bộ trưởng) cho đến khi nghỉ hưu năm 1987. Trước đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa từ năm 1955 đến năm 1976. Ông là vị Thủ tướng Việt Nam tại vị lâu nhất (1955–1987). Ông là học trò, cộng sự </s>
<s>Giai đoạn năm 1955-1976, Phạm Văn Đồng nắm giữ chức vụ gì? </s></s>7. Trước đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa từ năm 1955 đến năm 1976. Ông là vị Thủ tướng Việt Nam tại vị lâu nhất (1955–1987). Ông là học trò, cộn

[0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3]

##### Converts character-level answer annotations into token-level positions

Since the task of the question answering model is to predict where the answer is located in the context by outputting the start and end token indices.

We will loop through chunks of tokenized context then check if the annotated answer is fully present in each tokenized chunk of the context.
* If the answer is fully within a chunk, it identifies the tokens where the answer starts and ends.
* If the answer is not fully present in a chunk, it assigns `(0, 0)` to indicate no valid answer in that chunk.

We need to find which is the case.


In [ ]:
answers = raw_datasets["train"][2:6]["answers"]
start_positions = []
end_positions = []

for i, offset in enumerate(inputs["offset_mapping"]):
  sample_idx = inputs["overflow_to_sample_mapping"][i]
  answer = answers[sample_idx]
  start_char = answer["answer_start"][0]
  end_char = answer["answer_start"][0] + len(answer["text"][0])
  sequence_ids = inputs.sequence_ids(i)

  # Find the start and end of the context
  idx = 0
  while sequence_ids[idx] != 1:
    idx += 1
  context_start = idx
  while sequence_ids[idx] == 1:
    idx += 1
  context_end = idx - 1

  # If the answer is not fully inside the context, label is (0, 0)
  if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
    start_positions.append(0)
    end_positions.append(0)
  else:
    # Otherwise it's the start and end token positions
    idx = context_start
    while idx <= context_end and offset[idx][0] <= start_char:
      idx += 1
    start_positions.append(idx - 1)
    idx = context_end
    while idx >= context_start and offset[idx][1] >= end_char:
      idx -= 1
    end_positions.append(idx + 1)

start_positions, end_positions

([85, 57, 29, 0, 25, 0, 0, 0, 0, 0, 0, 48, 29, 0, 0],
 [95, 67, 39, 0, 27, 0, 0, 0, 0, 0, 0, 85, 33, 0, 0])

In [ ]:
# check if the start and end positions are matched
# For the first feature we find (85, 95) as labels, so let’s compare the theoretical answer with the decoded span of tokens from 85 to 95

idx = 0
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

start = start_positions[idx]
end = end_positions[idx]
labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")

Theoretical answer: Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa, labels give: Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa


In [ ]:
# check for index 4, where we set the pair is (0, 0), which means the answer is not in that context chunk
idx = 4
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

decoded_example = tokenizer.decode(inputs["input_ids"][idx])
print(f"Theoretical answer: {answer}, decoded example: {decoded_example}")

Theoretical answer: Phạm Văn Đồng, decoded example: <s>Chủ tịch Hội đồng Bộ trưởng đầu tiên của nước Cộng hòa xã hội chủ nghĩa Việt Nam là ai? </s></s>Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4 năm 2000) là Thủ tướng đầu tiên của nước Cộng hòa Xã hội chủ nghĩa Việt Nam từ năm 1976 (từ năm 1981 gọi là Chủ tịch Hội đồng Bộ trưởng) cho đến khi nghỉ hưu năm 1987. Trước đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hò</s>


##### Turn everything into a function to apply on the whole training dataset

In [ ]:
max_length = 384
stride = 128

def preprocess_training_dataset(examples):
  questions = [q.strip() for q in examples["question"]]
  inputs = tokenizer(
      questions,
      examples["context"],
      max_length=max_length,
      truncation="only_second",
      stride=stride,
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
      padding="max_length",  
  )

  offset_mapping = inputs.pop("offset_mapping")
  sample_map = inputs.pop("overflow_to_sample_mapping")
  answers = examples["answers"]
  is_impossible = examples["is_impossible"] 
  start_positions = []
  end_positions = []

  for i, offset in enumerate(offset_mapping):
      sample_idx = sample_map[i]

      # Check if the example is impossible and set answers accordingly
      if is_impossible[sample_idx]:
          answer = {"text": examples["plausible_answers"][sample_idx], "answer_start": [0]}
      else:
          answer = answers[sample_idx]

      start_char = answer["answer_start"][0]
      end_char = answer["answer_start"][0] + len(answer["text"][0])
      sequence_ids = inputs.sequence_ids(i)

      # Find the start and end of the context
      idx = 0
      while sequence_ids[idx] != 1:
          idx += 1
      context_start = idx
      while sequence_ids[idx] == 1:
          idx += 1
      context_end = idx - 1

      # If the answer is not fully inside the context, label is (0, 0)
      if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
          start_positions.append(0)
          end_positions.append(0)
      else:
          # Otherwise it's the start and end token positions
          idx = context_start
          while idx <= context_end and offset[idx][0] <= start_char:
              idx += 1
          start_positions.append(idx - 1)

          idx = context_end
          while idx >= context_start and offset[idx][1] >= end_char:
              idx -= 1
          end_positions.append(idx + 1)

  inputs["start_positions"] = start_positions
  inputs["end_positions"] = end_positions
  return inputs

In [ ]:
# apply the function to the whole training dataset
train_dataset = raw_datasets["train"].map(
    preprocess_training_dataset,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)
len(raw_datasets["train"]), len(train_dataset)

Map:   0%|          | 0/19238 [00:00<?, ? examples/s]

(19238, 20530)

### Proprocessing the validation data

#### A tiny bit of cleanup of the offset mappings. They will contain offsets for the question and the context, but once we’re in the post-processing stage we won’t have any way to know which part of the input IDs corresponded to the context and which part was the question. So, we’ll set the offsets corresponding to the question to None

In [ ]:
def preprocess_validation_dataset(examples):
  questions = [q.strip() for q in examples["question"]]
  inputs = tokenizer(
      questions,
      examples["context"],
      max_length=max_length,
      truncation="only_second",
      stride=stride,
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
      padding="max_length",
  )

  sample_map = inputs.pop("overflow_to_sample_mapping")
  example_ids = []
  is_impossible = examples["is_impossible"]
  plausible_answers = examples["plausible_answers"]

  for i in range(len(inputs["input_ids"])):
      sample_idx = sample_map[i]
      example_ids.append(examples["id"][sample_idx])

      # Check if the example is impossible and set answers accordingly
      if is_impossible[sample_idx]:
          answer = {"text": plausible_answers[sample_idx], "answer_start": [0]}
      else:
          answer = examples["answers"][sample_idx]

      sequence_ids = inputs.sequence_ids(i)
      offset = inputs["offset_mapping"][i]
      inputs["offset_mapping"][i] = [
          o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
      ]

  inputs["example_id"] = example_ids
  return inputs

In [ ]:
# apply to the whole validation set
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_dataset,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)
len(raw_datasets["validation"]), len(validation_dataset)

Map:   0%|          | 0/3814 [00:00<?, ? examples/s]

(3814, 3936)

### Compute the metrics

In [ ]:
def compute_metrics(start_logits, end_logits, features, examples):
    import collections
    from tqdm.auto import tqdm
    import numpy as np
    import evaluate

    n_best = 20
    max_answer_length = 200
    metric = evaluate.load("squad")

    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        is_impossible = example.get("is_impossible", False) 
        plausible_answers = example.get("plausible_answers", [])

        answers = []
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index].get("offset_mapping", [])

            if not offsets:
                continue

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()

            for start_index in start_indexes:
                for end_index in end_indexes:
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            # Handle impossible questions by selecting a plausible answer or an empty string
            if is_impossible and plausible_answers:
                predicted_answers.append(
                    {"id": example_id, "prediction_text": plausible_answers[0]["text"]}
                )
            else:
                predicted_answers.append({"id": example_id, "prediction_text": ""})

    # Create references for metric computation
    theoretical_answers = [
        {
            "id": ex["id"],
            "answers": ex["plausible_answers"] if ex.get("is_impossible", False) else ex["answers"],
        }
        for ex in examples
    ]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)


### Train the model

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoModel
model = AutoModelForQuestionAnswering.from_pretrained("vinai/bartpho-word-base")

pytorch_model.bin:   0%|          | 0.00/600M [00:00<?, ?B/s]

Some weights of MBartForQuestionAnswering were not initialized from the model checkpoint at vinai/bartpho-word-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import Trainer, TrainingArguments
epochs = 7  #5, 10
batch_size = 16   
lr = 2e-5

training_args = TrainingArguments(
    output_dir = "checkpoints",
    eval_strategy = "steps", # print evaluation after finishing an epoch
    num_train_epochs=epochs,
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_total_limit=1,
    save_steps=2000, 
    eval_steps=2000,
    gradient_accumulation_steps=2,
    eval_accumulation_steps=2,
    hub_model_id="REPO_NAME",    # huggingface repo name to upload to
    hub_token="YOUR_HF_TOKEN",   # your huggingface token
)

In [ ]:
model.resize_token_embeddings(len(tokenizer))


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


MBartScaledWordEmbedding(66120, 768, padding_idx=1)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics
)
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss,Validation Loss
2000,1.452900,No log
4000,0.912100,No log


TrainOutput(global_step=4494, training_loss=1.5002847492403064, metrics={'train_runtime': 12148.9549, 'train_samples_per_second': 11.829, 'train_steps_per_second': 0.37, 'total_flos': 3.28609863122688e+16, 'train_loss': 1.5002847492403064, 'epoch': 7.0})

In [ ]:
predictions, _, _ = trainer.predict(validation_dataset)
print(type(predictions), predictions)


<class 'tuple'> (array([[ -3.3133116, -11.851762 , -10.032443 , ..., -16.62144  ,
        -16.537405 , -16.162909 ],
       [ -3.279092 , -11.100981 , -12.605898 , ..., -16.00767  ,
        -15.932522 , -15.6761   ],
       [ -3.1924262, -12.716864 , -16.498486 , ..., -15.910229 ,
        -15.802593 , -15.54617  ],
       ...,
       [ -3.167385 , -12.628684 , -17.220419 , ..., -16.407154 ,
        -16.376526 , -16.15847  ],
       [ -3.0042617, -14.146257 , -15.430178 , ..., -16.2899   ,
        -16.241009 , -15.922951 ],
       [ -3.1889284, -14.8051605, -16.00679  , ..., -16.346695 ,
        -16.302876 , -16.127335 ]], dtype=float32), array([[ -3.5506155, -16.137974 , -14.847229 , ..., -15.138203 ,
        -15.248995 , -15.5822315],
       [ -3.5151772, -17.865335 , -15.692579 , ..., -15.644508 ,
        -15.741877 , -15.830298 ],
       [ -3.5107715, -19.076933 , -15.062859 , ..., -15.586006 ,
        -15.700405 , -15.815512 ],
       ...,
       [ -3.0781603, -17.358414 , -15.2066

In [ ]:

# Unpack predictions based on structure
if isinstance(predictions, tuple):
    start_logits = predictions[0]
    end_logits = predictions[1]
elif isinstance(predictions, dict):
    # Some models may return predictions as a dict
    start_logits = predictions['start_logits']
    end_logits = predictions['end_logits']
else:
    raise ValueError("Unexpected predictions format!")

In [ ]:
compute_metrics(start_logits, end_logits, validation_dataset, raw_datasets["validation"])

  0%|          | 0/3814 [00:00<?, ?it/s]

{'exact_match': 49.18720503408495, 'f1': 71.06630108926625}

In [ ]:
import gc
gc.collect()

2305

In [ ]:
import torch
torch.cuda.empty_cache()


In [ ]:
!pip install transformers huggingface-hub


In [ ]:
trainer.push_to_hub(commit_message="Training completed")

model.safetensors:   0%|          | 0.00/606M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

events.out.tfevents.1737610045.867b10aaee61.515.0:   0%|          | 0.00/7.86k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/lizz4rd/bartpho-word-base-question-answering/commit/9605e324abf7e661e9e6a0db9ac511a4197e1a38', commit_message='Training completed', commit_description='', oid='9605e324abf7e661e9e6a0db9ac511a4197e1a38', pr_url=None, repo_url=RepoUrl('https://huggingface.co/lizz4rd/bartpho-word-base-question-answering', endpoint='https://huggingface.co', repo_type='model', repo_id='lizz4rd/bartpho-word-base-question-answering'), pr_revision=None, pr_num=None)